# ПРОЕКТ АНАЛИЗ БАЗЫ ДАННЫХ ПРИ ПОМОЩИ SQL ЗАПРОСОВ

# Цели исследования

Анализ базы данных с информацией о книгах, издательствах, авторах и пользовательских обзорах на книги с целью формулирования ценного предложения для нового продукта компании по чтению книг по подписке

# Импорт необходимых библиотек и подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
              'pwd': 'Sdf4$2;d-d30pp', # пароль
              'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
              'port': 6432, # порт подключения
              'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'], db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Знакомство с данными

Посмотрим на таблицу с данными о книгах - books

Она содержит: 

book_id — идентификатор книги

author_id — идентификатор автора

title — название книги

num_pages — количество страниц

publication_date — дата публикации книги

publisher_id — идентификатор издателя

Выведем первые 5 строк таблицы

In [2]:
query = '''
            SELECT * 
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Теперь взглянем на таблицу с данными об авторах - authors

Она содержит:

author_id — идентификатор автора

author — имя автора

Выведем первые 5 строк таблицы

In [3]:
query = '''
            SELECT * 
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Изучим таблицу с данными об издательствах - publishers

Она содержит:

publisher_id — идентификатор издательства

publisher — название издательства

Выведем первые 5 строк таблицы

In [4]:
query = '''
            SELECT * 
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Следующая таблица - с данными о пользовательских оценках книг - ratings

Она содержит:

rating_id — идентификатор оценки

book_id — идентификатор книги

username — имя пользователя, оставившего оценку

rating — оценка книги

Выведем первые 5 строк таблицы

In [5]:
query = '''
            SELECT * 
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Последняя таблица - reviews - содержит данные о пользовательских обзорах на книги

Она содержит:

review_id — идентификатор обзора

book_id — идентификатор книги

username — имя пользователя, написавшего обзор

text — текст обзора

Выведем первые 5 строк таблицы

In [6]:
query = '''
            SELECT * 
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Итак, мы подключились к базе данных, выгрузили 5 таблиц и готовы отвечать на поставленные вопросы

# Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
            SELECT COUNT(book_id)
            FROM books
            WHERE publication_date>'01-01-2000'
        '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


Как мы видим, после 1 января 2000 года вышло в свет 819 книг

# Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''
            SELECT b.book_id,
                   b.title,
                   COUNT(DISTINCT(rev.review_id)) AS count_of_reviews,
                   AVG(rat.rating) AS average_rating      
            FROM books AS b
            LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
            LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
            GROUP BY b.book_id
            ORDER BY count_of_reviews
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_of_reviews,average_rating
0,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
1,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
2,221,Essential Tales and Poems,0,4.000000
3,808,The Natural Way to Draw,0,3.000000
4,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
...,...,...,...,...
995,733,The Giver (The Giver #1),6,3.750000
996,854,The Road,6,3.772727
997,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
998,696,The Da Vinci Code (Robert Langdon #2),6,3.830508


Как мы видим, есть книги, к которым не было написано ни одного пользовательских обзора

Есть один лидер по количеству обзоров - книга с id 948, на нее написано 7 обзоров

Эта книга называется Twilight, в переводе на русский - Сумерки, первая часть

Это действительно очень известное произведение

Но средняя оценка у этой книги - лишь 3.6


# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
query = '''
            SELECT pub.publisher,
                   COUNT(book_id) AS cnt
            FROM publishers AS pub
            LEFT JOIN books AS b ON pub.publisher_id=b.publisher_id
            WHERE b.num_pages>50
            GROUP BY pub.publisher
            ORDER BY cnt DESC
            LIMIT 1
            
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,cnt
0,Penguin Books,42


Мы выяснили, что издательство, выпустившее наибольшее число книг толще 50 страниц, носит название Penguin Books

Это британское издательство, дополнительную информацию про него при необходиомсти можно изучить в Википедии

https://ru.wikipedia.org/wiki/Penguin_Books

Вероятно, это действительно популярное издательство и нашей компании стоит поподробнее изучить его

# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query = '''
            SELECT a.author,
                   AVG(rat.rating) AS average_rating
            FROM authors AS a
            LEFT JOIN books AS b ON a.author_id=b.author_id
            LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
            GROUP BY a.author
            HAVING COUNT(rat.rating)>50
            ORDER BY AVG(rat.rating) DESC
            LIMIT 1
            
            
        '''
pd.io.sql.read_sql(query, con = engine)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.288462


Как мы видим, автором с самой высокой средней оценкой книг является Джоан Роулинг, известная всему миру автор бестселлеров про Гарри Поттера

В нашем случае она представлена, вероятно, в соавторстве с иллюстратором Мэри Гранпрэ, как раз автором иллюстраций к книгам о Поттере

Резуьтат в целом довольно закономерен, ведь произведение о юном волшебнике действительно является одной из самых известных книг для любого возраста

# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query = '''
            SELECT AVG(count_reviews) AS average_count_of_reviews
            FROM (
            SELECT username, COUNT(*) count_reviews
            FROM reviews
            WHERE username IN
            (
            SELECT username--, COUNT(*)
            FROM ratings
            GROUP BY username
            HAVING COUNT(*) > 50
            )
            GROUP BY username) review
                      
        '''
pd.io.sql.read_sql(query, con = engine)

,average_count_of_reviews
0,24.333333


Мы выяснили, что среднее количество обзоров от пользователей, которые поставили больше 50 оценок - чуть более 24

То есть, можем предположить, что люди, которые ставят большое количество оценок, пишут и большое количество обзоров на литературу

Вероятно, это настоящие книголюбы

# ВЫВОДЫ

Анализ базы данных показал следующее: 

* После 1 января 2000 года вышло 819 новых книг

* Книга с наибольшим количеством обзоров - сага Сумерки. Однако, она имеет не слишком высокую среднюю оценку - 3.6

* Издательство, выпустившее наибольшее число книг толще 50 страниц, носит название Penguin Books

* Автором с самой высокой средней оценкой книг является Джоан Роулинг, известная всему миру автор бестселлеров про Гарри Поттера

* Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - чуть более 24
